In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.storage import LocalFileStore
from langchain.callbacks import StreamingStdOutCallbackHandler

# Caching for embeddings
cache_dir = LocalFileStore("./.cache/")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_length=20,
    max_token_limit=100,
    return_messages=True,
)
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# Loading and splitting the document
loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)


# Setting up embeddings with caching
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# Creating the vector store and retriever FAISS | Chroma (switching)
vector_store = FAISS.from_documents(docs, cached_embeddings)
retriver = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.
            
            Context: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)


chain = ({ "context": retriver,"question": RunnablePassthrough(),} | prompt | chat)

chain.invoke("Describe Victory Mansions") 

Victory Mansions is a place where Winston resides. It is described as a dilapidated apartment building in a poor state of repair. The building is run-down, with shabby walls and broken elevators. The living conditions are grim, with shortages of basic necessities like electricity and plumbing. The atmosphere is oppressive, with constant surveillance and a sense of fear and control. Despite its name, Victory Mansions do not reflect victory or luxury but rather symbolize the harsh and austere reality of life under the Party's rule.

AIMessageChunk(content="Victory Mansions is a place where Winston resides. It is described as a dilapidated apartment building in a poor state of repair. The building is run-down, with shabby walls and broken elevators. The living conditions are grim, with shortages of basic necessities like electricity and plumbing. The atmosphere is oppressive, with constant surveillance and a sense of fear and control. Despite its name, Victory Mansions do not reflect victory or luxury but rather symbolize the harsh and austere reality of life under the Party's rule.")

In [6]:
chain.invoke("Is Aaronson guilty?")

Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.

AIMessageChunk(content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [7]:
chain.invoke("What message did he write in the table?")

He wrote "2+2=5" on the table.

AIMessageChunk(content='He wrote "2+2=5" on the table.')

In [8]:
chain.invoke("Who is Julia?")

Julia is a character in the text who is significant to the protagonist.

AIMessageChunk(content='Julia is a character in the text who is significant to the protagonist.')

In [9]:
chain.invoke("Describe Victory Mansions")

Victory Mansions is a place where Winston resides. It is described as a dilapidated apartment building in a rundown area. The building is poorly maintained, with shabby walls and broken elevators. The atmosphere is grim and depressing, reflecting the overall bleakness of the society in which Winston lives.

AIMessageChunk(content='Victory Mansions is a place where Winston resides. It is described as a dilapidated apartment building in a rundown area. The building is poorly maintained, with shabby walls and broken elevators. The atmosphere is grim and depressing, reflecting the overall bleakness of the society in which Winston lives.')